# Классификация интентов
В этом ноутбуке произведем классификацию интентов

Скачаем данные и установим необходимое окружение:

In [1]:
! wget https://huggingface.co/datasets/blinoff/medical_qa_ru_data/resolve/main/medical_qa_ru_data.csv
! wget https://github.com/cimm-kzn/RuDReC/raw/master/data/rudrec_annotated.json
! pip install corus

--2022-10-11 13:39:59--  https://huggingface.co/datasets/blinoff/medical_qa_ru_data/resolve/main/medical_qa_ru_data.csv
Resolving huggingface.co (huggingface.co)... 44.195.102.200, 52.5.54.249, 54.173.5.192, ...
Connecting to huggingface.co (huggingface.co)|44.195.102.200|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/datasets/blinoff/medical_qa_ru_data/6aca4fb6dae499745c9ca1dfad8a0ade809573daf228c923fdf846ba4a639b50?response-content-disposition=attachment%3B%20filename%3D%22medical_qa_ru_data.csv%22&Expires=1665754800&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL2RhdGFzZXRzL2JsaW5vZmYvbWVkaWNhbF9xYV9ydV9kYXRhLzZhY2E0ZmI2ZGFlNDk5NzQ1YzljYTFkZmFkOGEwYWRlODA5NTczZGFmMjI4YzkyM2ZkZjg0NmJhNGE2MzliNTA~cmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj1hdHRhY2htZW50JTNCJTIwZmlsZW5hbWUlM0QlMjJtZWRpY2FsX3FhX3J1X2RhdGEuY3N2JTIyIiwiQ29uZGl0aW9uIjp7IkRhdGVMZXNzVGhhbiI6eyJBV1M6RXBvY2hUaW1lIjoxNjY1NzU0ODAwfX19X

In [2]:
!pip install -q transformers

     |████████████████████████████████| 5.3 MB 33.0 MB/s 
     |████████████████████████████████| 7.6 MB 53.8 MB/s 
     |████████████████████████████████| 163 kB 70.7 MB/s 


In [17]:
import numpy as np
import pandas as pd
import re
from corus import load_rudrec
from tqdm import tqdm_notebook
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, classification_report

Примонтируем гугл-диск и скачаем файл с вопросами и ответами мейла:

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
!ls "/content/drive/My Drive/GeekBrains/NLP/Course Project/"

IntentClassifier.ipynb	only_questions.txt  Otvety.txt


In [5]:
path =  "/content/drive/My Drive/GeekBrains/NLP/Course Project/"

Так как этот файл мы используем лишь для классификации интентов, сохраним только вопросы:

In [18]:
assert True

#Small preprocess of the answers

written = False

with open(path + "only_questions.txt", "w") as fout:
    with open(path + "Otvety.txt", "r") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written:
                fout.write(line.replace("\t", " ").strip() + "\n")
                written = True
                continue

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


0it [00:00, ?it/s]

Обернем 1-й интент в датасет и назначим ему класс 0:

In [15]:
data_1 = {'id': [], 'text': [], 'class': []}
with open(path + "only_questions.txt", "r") as f:
    for idx, line in enumerate(f):
      data_1['id'].append(idx)
      data_1['text'].append(line.strip())
      data_1['class'].append(0)
data_1 = pd.DataFrame(data_1)

In [16]:
data_1

,id,text,class
0,0,,0
1,1,вопрос о ТДВ)) давно и хорошо отдыхаем)) ЛИЧНО...,0
2,2,Как парни относятся к цветным линзам? Если у д...,0
3,3,"Что делать, сегодня нашёл 2 миллиона рублей? .",0
4,4,Эбу в двенашке называется Итэлма что за эбу? .,0
...,...,...,...
1163417,1163417,Между словами ПРЕЗИДЕНТ и РЕЗИДЕНТ есть что ли...,0
1163418,1163418,"""Если это мое, то оно никуда от меня не денетс...",0
1163419,1163419,А Вы халяву любите или совесть имеете???) .,0
1163420,1163420,Так много разных гороскопов кто-нибудь может п...,0


Посмотрим датасет для второго интента:



In [17]:
data_2 = pd.read_csv('medical_qa_ru_data.csv')
data_2

,date,categ,theme,desc,ans,spec10
0,"8 Октября 2017, 11:55",Оториноларингология,Применение Ларипронта.,"Ларипронт 20 талеток,через каждые 2-3 часа.Оче...",Что вы им лечите? Длительность приема Ларипрон...,Отоларинголог
1,"20 Февраля 2019, 13:24",Акушерство,Беременность,"Здравствуйте, я на 7-8 неделе беременности. С ...","Здравствуйте, это может быть признаком раннего...",NaN
2,"17 Марта 2015, 18:31",Другое,гинекология,Здравствуйте месячные должны придти 23 марта в...,Выполните исследование хгч,NaN
3,"13 Января 2019, 19:38",Терапия,Занятия спорта после сдачи крови,"Завтра иду с утра сдавать кровь ТТГ, Т4СВ, Кал...","Можно.;\nЗдравствуйте , да, попейте сладкого ч...",Терапевт
4,"28 Ноября 2017, 21:58",Другое,Таблетки,Мне прописали пить Аллохол. Врач написала пить...,Препарат принимается после еды. Уточните это ...,NaN
...,...,...,...,...,...,...
190330,"31 Июля 2014, 11:31",Офтальмология,Отлетела в глаз пластмасса,"Здравствуйте! В глаз отлетел кусок пластмассы,...","Ромашка есть дома, альбуцид;\nМожно промыть лю...",Офтальмолог
190331,"16 Марта 2014, 21:52",Другое,здравствуйте скажите пожалуста где я могу посм...,NaN,"В ""консультанте""",NaN
190332,"22 Ноября 2018, 11:02",Общая хирургия,палец,"Здравствуйте, три дня назад порезала палец бу...","Здравствуйте , пока обработать Хлоргексидином ...",Хирург
190333,"2 Ноября 2018, 21:19",Эндокринология,Эндокринолог прописал пить сиофор 500 при саха...,NaN,Такие препараты принимаются постоянно;\nЗдравс...,NaN


Из ответов оставим только первый ответ. Сообщения разделены ';\n'

In [18]:
data_2.ans = data_2.ans.apply(lambda x: re.split(r';\n', str(x))[0]) 
data_2.ans[0]

'Что вы им лечите? Длительность приема Ларипронта индивидуальна и устанавливается лечащим врачом.'

Из датасета оставим только столбцы 'desc' и 'ans', удалим строки с пустыми значениями, и назначим данному интенту класс 1:

In [19]:
data_2 = data_2[['desc', 'ans']]
data_2.dropna(subset=['desc', 'ans'], inplace=True)
data_2['class'] = 1
data_2

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,desc,ans,class
0,"Ларипронт 20 талеток,через каждые 2-3 часа.Оче...",Что вы им лечите? Длительность приема Ларипрон...,1
1,"Здравствуйте, я на 7-8 неделе беременности. С ...","Здравствуйте, это может быть признаком раннего...",1
2,Здравствуйте месячные должны придти 23 марта в...,Выполните исследование хгч,1
3,"Завтра иду с утра сдавать кровь ТТГ, Т4СВ, Кал...",Можно.,1
4,Мне прописали пить Аллохол. Врач написала пить...,Препарат принимается после еды. Уточните это ...,1
...,...,...,...
190328,"Не курю и не пью. Веду здоровый образ жизни, м...",В первую очередь вам необходимо обследовать пе...,1
190329,"Здравствуйте, очень прошу консультации. В лево...",Начинать следует с терапевтических методов. Ес...,1
190330,"Здравствуйте! В глаз отлетел кусок пластмассы,...","Ромашка есть дома, альбуцид",1
190332,"Здравствуйте, три дня назад порезала палец бу...","Здравствуйте , пока обработать Хлоргексидином ...",1


Теперь возьмем датасет для 3-го интента - rudrec_annotated. Берем только текст - отзывы на лекарства.

In [20]:
path = 'rudrec_annotated.json'
records = load_rudrec(path)

In [21]:
data_3 = {'id': [], 'text': [], 'class': []}
for idx, rec in enumerate(records):

    data_3['id'].append(idx)
    data_3['text'].append(rec.text.strip())
    data_3['class'].append(2)

data_3 = pd.DataFrame(data_3)

In [22]:
data_3

,id,text,class
0,0,"нам прописали, так мой ребенок сыпью покрылся,...",2
1,1,Общее впечатление : не подошел,2
2,2,Пила этот препарат для повышения иммунитета 5 ...,2
3,3,"Так как начала работать в аптеке, начала часто...",2
4,4,В месяц по нескольку раз причем со всеми вытек...,2
...,...,...,...
4804,4804,"Покраснение спадает, неприятные ощущения прохо...",2
4805,4805,"Из своего опыта отметил бы, что средство себя ...",2
4806,4806,Из своего опыта хотел бы порекомендовать начат...,2
4807,4807,Тогда оно может спасти от длительной болезни.,2


Третий датасет самый маленький - всего 4809 строк. Возьмем из остальных датасетов случайным семплированием такое же количество строк и объеденим в один датасет.

In [23]:
data_1 = data_1.sample(4809)
data_1

,id,text,class
1083604,1083604,"Патер Дий Александер (Хиневич, Александр Юрьев...",0
225453,225453,В коком веке родились денозавры .,0
937594,937594,Ответьте на следующие вопросы утвердительно ил...,0
593610,593610,Какой самый дешевый сайт для покупки тачскрина...,0
1094720,1094720,195 пожалуйста! срочно.,0
...,...,...,...
221065,221065,O eiai anou eaeay-ieaoau eioi? iaoey i iaaeoa ...,0
505299,505299,Кто купит сумки? .,0
695077,695077,трещина!! ( а не царапина. учтите! ) на экране...,0
403751,403751,Подскажите магазины или места в Москве? Живу в...,0


In [24]:
data_1 = data_1[['text', 'class']]

In [25]:
data_2 = data_2.sample(4809)

In [26]:
data_2

,desc,ans,class
65488,"Здравствуйте, в июле после лечения антибиотика...",У вас возможны кандидоз или люба другая инфекц...,1
75295,"Здравствуйте. Купался в реке, затянуло течение...",Преходящее нарушение питания мозга,1
136823,"Добрый день, у меня были проблемы с носом всег...","Да,осложнения не исключены,вам стоит показатьс...",1
107933,Здравствуйте! Писала недавно вопрос о задержке...,Здравствуйте! Да - ответ на оба вопроса.,1
131547,"29,178,78 начал болеть гайморит , температура ...",Что вас сейчас беспокоит? Каковы ваши жалобы?,1
...,...,...,...
142097,здравствуйте. мне 16 лет. у меня эрозия. мне н...,"Вероятно это продолжительность лечения, 10 дне...",1
175054,Может ли болеть живот от молочного шоколада?по...,разумеется может. не советую экспериментировать,1
122363,Здравствуйте. Диагноз F 23.1 мне в последнее в...,Возможно это связано с вашим заболеванием.,1
28538,Здравствуйте! Скажите мне прописали уколы в/м ...,"Здравствуйте,они никак не действуют на сердце,...",1


In [27]:
data_2_tmp = data_2[['desc', 'class']]
data_2_tmp.rename(columns={'desc': 'text'}, inplace=True)
data_2_tmp

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,text,class
65488,"Здравствуйте, в июле после лечения антибиотика...",1
75295,"Здравствуйте. Купался в реке, затянуло течение...",1
136823,"Добрый день, у меня были проблемы с носом всег...",1
107933,Здравствуйте! Писала недавно вопрос о задержке...,1
131547,"29,178,78 начал болеть гайморит , температура ...",1
...,...,...
142097,здравствуйте. мне 16 лет. у меня эрозия. мне н...,1
175054,Может ли болеть живот от молочного шоколада?по...,1
122363,Здравствуйте. Диагноз F 23.1 мне в последнее в...,1
28538,Здравствуйте! Скажите мне прописали уколы в/м ...,1


In [28]:
data_3 = data_3[['text', 'class']]

In [29]:
data = pd.concat([data_1, data_2_tmp, data_3], ignore_index=True)
data

,text,class
0,"Патер Дий Александер (Хиневич, Александр Юрьев...",0
1,В коком веке родились денозавры .,0
2,Ответьте на следующие вопросы утвердительно ил...,0
3,Какой самый дешевый сайт для покупки тачскрина...,0
4,195 пожалуйста! срочно.,0
...,...,...
14422,"Покраснение спадает, неприятные ощущения прохо...",2
14423,"Из своего опыта отметил бы, что средство себя ...",2
14424,Из своего опыта хотел бы порекомендовать начат...,2
14425,Тогда оно может спасти от длительной болезни.,2


In [30]:
data = data.sample(frac=1).reset_index(drop=True)
data

,text,class
0,Пропилеи.,0
1,"На приеме у врача мне было даже неловко, когда...",2
2,Сейчас очень много всяких продуктов помогающих...,2
3,Вчера выпивал сильно правая нга болит ниже колена,1
4,Возможно покраснение в течении нескольких дней...,2
...,...,...
14422,"Здравствуйте, не подскажете, смогу ли я оценит...",1
14423,Был хронический простатит.\n \n ...,1
14424,У других членов семьи никаких последствий.,2
14425,где можно скачать сериал Доктор Кто на француз...,0


Сохраним датасеты для последующей работы:

In [31]:
path =  "/content/drive/My Drive/GeekBrains/NLP/Course Project/"
data.to_csv(path + 'common_data.csv', encoding='utf-8', index=False)
data_2.to_csv(path + 'qa_medical.csv', encoding='utf-8', index=False)
data_3.to_csv(path + 'drugs_feedback.csv', encoding='utf-8', index=False)

In [6]:
data = pd.read_csv(path+'common_data.csv')
data

,text,class
0,Пропилеи.,0
1,"На приеме у врача мне было даже неловко, когда...",2
2,Сейчас очень много всяких продуктов помогающих...,2
3,Вчера выпивал сильно правая нга болит ниже колена,1
4,Возможно покраснение в течении нескольких дней...,2
...,...,...
14422,"Здравствуйте, не подскажете, смогу ли я оценит...",1
14423,Был хронический простатит.\n \n ...,1
14424,У других членов семьи никаких последствий.,2
14425,где можно скачать сериал Доктор Кто на француз...,0


Разобьем датасет на тренировочную и валидационную выборку:

In [7]:
text_train, text_valid, y_train, y_valid = train_test_split(data['text'], 
                                                            data['class'], 
                                                            test_size=.2, 
                                                            shuffle=True,
                                                            stratify=data['class'],
                                                            random_state=42)

Загрузим мультилингвальную модель для классификации с количеством классов, равным 3:

In [8]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=3)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Токенизируем датасет и создадим датасет для обучения и валидации:

In [9]:
tf_train = tokenizer(text_train.tolist(), padding="max_length", truncation=True, return_tensors='tf')

In [10]:
tf_valid = tokenizer(text_valid.tolist(), padding="max_length", truncation=True, return_tensors='tf')

In [11]:
num_classes = 3
tf_y_train = tf.keras.utils.to_categorical(y_train.tolist(), num_classes)
tf_y_valid = tf.keras.utils.to_categorical(y_valid.tolist(), num_classes)

In [12]:
train_features = {x: tf_train[x] for x in tokenizer.model_input_names}
train_tf_dataset = tf.data.Dataset.from_tensor_slices((train_features, tf_y_train))
train_tf_dataset = train_tf_dataset.shuffle(len(tf_train)).batch(4)

valid_features = {x: tf_valid[x] for x in tokenizer.model_input_names}
valid_tf_dataset = tf.data.Dataset.from_tensor_slices((valid_features, tf_y_valid))
valid_tf_dataset = valid_tf_dataset.batch(4)

Заморозим бертовый слой, оставив для обучения только слой классификации, и скомпилируем модель:

In [13]:
model.layers[0].trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=[tf.metrics.CategoricalAccuracy('accuracy')],
)

model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  177853440 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  2307      
                                                                 
Total params: 177,855,747
Trainable params: 2,307
Non-trainable params: 177,853,440
_________________________________________________________________


Обучим модель:

In [14]:
model.fit(train_tf_dataset, validation_data=valid_tf_dataset, epochs=3)

Epoch 1/3
2886/2886 [==============================] - 740s 251ms/step - loss: 0.5848 - accuracy: 0.7998 - val_loss: 0.3400 - val_accuracy: 0.8957
Epoch 2/3
2886/2886 [==============================] - 723s 251ms/step - loss: 0.3948 - accuracy: 0.8647 - val_loss: 0.2796 - val_accuracy: 0.9085
Epoch 3/3
2886/2886 [==============================] - 725s 251ms/step - loss: 0.3573 - accuracy: 0.8727 - val_loss: 0.2442 - val_accuracy: 0.9158


In [15]:
model.fit(train_tf_dataset, validation_data=valid_tf_dataset, epochs=1)

2886/2886 [==============================] - 729s 253ms/step - loss: 0.3427 - accuracy: 0.8777 - val_loss: 0.2431 - val_accuracy: 0.9130


Сохраним модель и токенайзер:

In [16]:
tokenizer.save_pretrained(path + 'tokenizer_intent_classifier/')
model.save_pretrained(path + 'model_intent_classifier/')

Функция обратного вызова для построения отчета классификации при валидации:

In [18]:
class Metrics(Callback):


    def on_test_end(self, epoch, logs=None):
        val_predict = model.predict(valid_tf_dataset).logits.argmax(axis=1)
        print('\n', classification_report(y_valid.astype('int32'), val_predict))

Построим отчет классификации на валидационном датасете:

In [20]:
model.evaluate(valid_tf_dataset, callbacks=[Metrics()])

722/722 [==============================] - ETA: 0s - loss: 0.2431 - accuracy: 0.9130
               precision    recall  f1-score   support

           0       0.95      0.85      0.89       962
           1       0.92      0.94      0.93       962
           2       0.88      0.95      0.91       962

    accuracy                           0.91      2886
   macro avg       0.92      0.91      0.91      2886
weighted avg       0.92      0.91      0.91      2886

722/722 [==============================] - 260s 360ms/step - loss: 0.2431 - accuracy: 0.9130


[0.24309058487415314, 0.9130284190177917]

**Вывод по классификатору:**
Я ожидал, что модель будет сильно путать 1-й и 2-й классы (обращения к врачу и отзывы на лекарства), но модель с хорошей точностью разделяет все три интента.

In [76]:
tokenizer1 = AutoTokenizer.from_pretrained(path + 'tokenizer_intent_classifier/')
model1 = TFAutoModelForSequenceClassification.from_pretrained(path + 'model_intent_classifier/')

Some layers from the model checkpoint at /content/drive/My Drive/GeekBrains/NLP/Course Project/model_intent_classifier/ were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/drive/My Drive/GeekBrains/NLP/Course Project/model_intent_classifier/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassifica

In [77]:
def intent_classifier(sentence):
    sent_token = tokenizer1(sentence, padding="max_length", truncation=True, return_tensors='tf')
    predict = model1(sent_token).logits.numpy()
    print(predict)
    return predict.argmax()

In [78]:
intent_classifier('У меня сильный насморк, подскажите, что делать?')

[[-5.5884027   3.6090636  -0.65658796]]


1

In [79]:
intent_classifier('Привет!)')

[[ 1.4303114 -3.0417693  0.8880274]]


0

In [80]:
intent_classifier('Привет!')

[[-2.3584783 -2.1112635  3.1485562]]


2

In [81]:
intent_classifier('Всем привет! Че молчим?')

[[-3.8528452   2.3096716  -0.25924304]]


1

In [82]:
intent_classifier('С фарингосептом стало полегче')

[[-2.787788   -0.15614022  1.4340233 ]]


2

In [83]:
intent_classifier('арбидол не помогает при простуде')

[[-2.6859856   1.193482    0.01684308]]


1

In [84]:
intent_classifier('Арбидол не помогает улучшить иммунитет')

[[-4.691154   1.0511379  1.4454162]]


2

Как видно по примерам, классификатор вроде неплохо работает. **Эти примеры я придумал, а не брал из датасета!** Но есть в его работе и нюансы:
- Видимо в болталке классификатор переобучился на смайлики. Без смайлика ни одно предложение не классифицирует как интент болталки. Ну тем и легче будет чат-бот направить в нужное русло.
- в реальности как я и предполагал первый и второй классы путаются - придумать предложение, которое распозналось бы как отзыв на лекарство оказалось непросто.